In [ ]:
import os
import sys


try:
    AUTO_DIR=os.environ["AUTO_DIR"]        
    sys.path.append(os.path.join(AUTO_DIR, "python") )
except KeyError:
    pass


In [ ]:
import auto

 AUTO Demo cusp
===============

Let us study the dynamical system 
\begin{equation}
\frac{d}{dt} x(t) = \mu + \lambda x - x^3
\end{equation}

Create the `cusp_c.c` file. Unhide the following cell to edit the source code.

In [ ]:
%%writefile cusp_c.c
#include "auto_f2c.h"

/* ---------------------------------------------------------------------- */
/* ---------------------------------------------------------------------- */
/*   cusp_c.c - cusp normal form                                          */
/* ---------------------------------------------------------------------- */
/* ---------------------------------------------------------------------- */

int func(integer ndim, const doublereal *u, const integer *icp, 
	const doublereal *par, integer ijac, doublereal *f, doublereal *dfdu, 
	doublereal *dfdp)
{

    doublereal x = u[0];
    doublereal lambda = par[0], mu = par[1];

    f[0] = mu + lambda*x - x*x*x;

    return 0;
} /* func */


int stpnt(integer ndim, doublereal t, doublereal *u, doublereal *par)
{
    /* Function Body */
    /* mu */
    par[0] = 1.0;
    /* lambda */
    par[1] = 0.0;

    /* x */
    u[0] = 0.0;
    return 0;
} /* stpnt */


/* Subroutine */ int pvls(integer ndim, const doublereal *u,
          doublereal *par)
{
    return 0;
} /* pvls */


/* Subroutine */ int bcnd(integer ndim, const doublereal *par, const integer *icp,
          integer nbc, const doublereal *u0, const doublereal *u1, integer ijac,
          doublereal *fb, doublereal *dbc)
{
    return 0;
} /* bcnd */


/* Subroutine */ int icnd(integer ndim, const doublereal *par, const integer *icp,
          integer nint, const doublereal *u, const doublereal *uold,
          const doublereal *udot, const doublereal *upold, integer ijac,
          doublereal *fi, doublereal *dint)
{
    return 0;
} /* icnd */


/* Subroutine */ int fopt(integer ndim, const doublereal *u, const integer *icp,
          const doublereal *par, integer ijac,
          doublereal *fs, doublereal *dfdu, doublereal *dfdp)
{
    return 0;
} /* fopt */


 Load the files cusp.c and c.cusp into the AUTO
 command interpreter.

In [ ]:
cusp = auto.load('cusp_c',
                parnames = {1:'lambda', 2:'mu'},
                unames = {1:'x'},
                NDIM=   1, IPS =   1, IRS =   0, ILP =   1,
                ICP =  ['mu', 'lambda'],
                NTST=   5, NCOL=   4, IAD =   3, ISP =   2, ISW = 1, IPLT= 0, NBC= 0, NINT= 0,
                NMX=  200, NPR=   20, MXBF=   0, IID =   2, ITMX= 8, ITNW= 5, NWTN= 3, JAC= 0,
                EPSL= 1e-06, EPSU = 1e-06, EPSS =0.0001,
                DS  =  0.01, DSMIN= 0.005, DSMAX=   0.1, IADS=   1,
                NPAR = 2, THL =  {}, THU =  {},
                UZSTOP = {'mu': [-2.0, 2.0]}
)

 Run and store the result in the Python variable mu

In [ ]:
mu = auto.run(cusp)

 Run backwards, and append to mu

In [ ]:
mu = mu + auto.run(cusp,DS='-')

 Relabel solutions

In [ ]:
mu = auto.relabel(mu)

 Save to b.mu, s.mu, and d.mu

In [ ]:
auto.save(mu,'mu')

 Plot bifurcation diagram
 
(Unhide the following cell to adjust the plotting defaults).

In [ ]:
%%writefile autorc
[AUTO_plotter]

#default_option="d1"
#d1 = {"grid": "no", "use_labels": 1, "use_symbols": 1, "stability": 1}
# similarly you can redefine d0, d2, d3, d4.

#grid = "no"
#stability = 0
#use_labels = 1
#use_symbols = 1

#top_title = ''
#top_title_fontsize = 12

#xlabel = ''
#xlabel_fontsize = 12
#ylabel = ''
#ylabel_fontsize = 12

#solution_indepvarname = "time"
#solution_coordnames = ["$x$","$y$","$z$"]
#bifurcation_coordnames = ["?", "L2-norm"]

#line_width = 2.0
#dashes = (6.0,6.0)
#background = "white"
#foreground = "black"
#color_list = "black red green blue"
#symbol_color = "red"
#symbol_font = "-misc-fixed-*-*-*-*-*-*-*-*-*-*-*-*"
#decorations = 1
#smart_label = 1
#minx = 0
#maxx = 0
#miny = 0
#maxy = 0
#width = 600
#height = 480
#left_margin = 80
#right_margin = 40
#top_margin = 40
#bottom_margin = 40
#xticks = 5
#yticks = 5
#tick_label_template = "%.2e"
#tick_length = 0.2
#odd_tick_length = 0.4
#even_tick_length = 0.2
#ps_colormode = "color"
#mark_t = None #or a real value between 0 and 1

#type = "bifurcation" # or "solution"

#bifurcation_x = [0]
#bifurcation_y = [1]

#solution_x = ["t"]
#solution_y = [0]

## Sets of columns that the user is likely to want to use
#bifurcation_column_defaults = None
#solution_column_defaults = None

# The label(s) of the solution we wish to draw
#label = [1,2,3]
#label_defaults = None
# The index/indices of the solution we wish to draw
#index = [0]

#bifurcation_diagram_filename = 'fort.7'
#solution_filename = 'fort.8'

#bifurcation_symbol = "square"
#limit_point_symbol = None
#hopf_symbol = "fillsquare"
#period_doubling_symbol = "doubletriangle"
#torus_symbol = "filldiamond"
#user_point_symbol = "U"
#error_symbol = None



In [ ]:
p = auto.plot(mu)
p.config(bifurcation_y=['x'])

 Set the new start label to the first LP label in b.mu and s.mu

In [ ]:
lp1 = auto.load(mu('LP1'), ISW=2)

 Continue from this label in two parameters

In [ ]:
cusp = auto.run(lp1)
cusp = cusp + auto.run(lp1,DS='-')

 save to b.cusp, s.cusp, and d.cusp

In [ ]:
auto.save(cusp,'cusp')

 Plot the cusp

In [ ]:
p = auto.plot(cusp)
p.config(bifurcation_y=['lambda'])

In [ ]:
# Stop automatic execution at this point
assert(False)

clean the directory

In [ ]:
# comment out to clean directory
auto.delete("mu")
auto.delete("cusp")
auto.clean()
!rm -f cusp_c.c autorc